## 1. (Computation) LU Factorization

We first implement the (strict) LU factorization.  You can start from the Matlab code for Gaussian elimination given on page 75 of the Sauer text, reproduced below.  In the code below, `a` is an n by n square matrix provided as input, `b` is a vector, though we will not be using it for the LU factorization.

```
for j = 1 : n-1
    if abs(a(j,j))<eps; error(’zero pivot encountered’); end
    for i = j+1 : n
        mult = a(i,j)/a(j,j);
        for k = j+1:n
            a(i,k) = a(i,k) - mult*a(j,k);
        end
        b(i) = b(i) - mult*b(j);
    end
end
```

1. Create a matrix L, and store the appropriate entries of $l_{i,j}$ to create the unit lower triangular factor.  

2. The code above does not zero out the entries below the diagonal.  Where is it safe to do so?  Add this in the loop.

3. Test your code on the matrix $A = \begin{bmatrix} 1 & 2 \\ 3 & 4 \end{bmatrix}.$ Make sure to take the resulting factors $L$ and $U$ and multiply them to show you get the original `a`.  Give a larger, 3 by 3 matrix and test your algorithm.

In [15]:
def lu(A):
    '''Computes the (strict) LU factorization of the matrix A'''
    L = 0
    U = 0
        
    
    return L, U


## 2. Operation Counts

We will count the floating point operations involved in the (strict) LU factorization algorithm.
We measure in terms of floating point operations, for example, the command 
`a(i,k) = a(i,k) - mult*a(j,k);` in the
innermost loop of the pseudocode has two operations, addition and multiplication, which
are counted equally.

1. (Theory) Compute the time of the inner loop in terms of $n$ and $j$,
by expressing it as a summation of the form:
$$\sum_{k = ?}^n 2 = \dots$$

2. (Theory) The outer two loops can be added as summations, so that the computational cost is a triple sum.  Write it out and compute.

3. (Computation) You can time your algorithm using 
```
    import timeit
    start_all = timeit.default_timer()
    # run code here
    stop_all = timeit.default_timer()
    print('Time: ', stop_all - start_all)  
```
You can create a random 100 by 100 matrix with the command `A = np.random.randn(100,100)` and test the time for computing `lu(A)`.  Double the matrix size $n$ and see how the computational time grows.  Note: depending on the platform you're using you may need to start with a smaller matrix to run in a reasonable amount of time.  

4. (Computation) Use at least four different matrix sizes, differing by a factor of 2 and compute the time scaling of your algorithm, presenting it as a table or a plot.

In [2]:
if __name__ == "__main__":
    pass
    # Code inside this block will be hidden from the autograder.

## 3. (Theory) Condition number vs determinant

Consider an $n \times n$ matrix of the form 
$$
A = \begin{bmatrix}
1 & -1 & -1 & \cdots & -1 \\
0 &  1 & -1 & \cdots & -1 \\
0 &  0 &  1 & \cdots & -1 \\
&\vdots  &&\ddots & \vdots \\
0 & 0 &   0 & \cdots & 1
\end{bmatrix}
$$
1. Compute the condition number of $A$ using the infinity norm $\| \cdot \|_\infty$.  
2. Compute the determinant of $A$ and $A^{-1}.$
3. Find an input right-hand side where there is low relative backward error, but high relative forward error.